## 라이브러리 설치

In [18]:
# Colab에 필요한 패키지 설치
#!pip install selenium
#!pip install webdriver-manager
#!pip install pandas
#!pip install sqlalchemy
#!pip install pymysql
#! pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.1 MB/s eta 0:00:00


In [ ]:
# 2. Chrome 브라우저 설치
!apt-get update  # 패키지 목록 업데이트
!apt-get install -y wget unzip  # wget과 unzip 패키지 설치
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -  # Google의 리눅스 서명 키 추가
!sh -c 'echo "deb [arch=amd64] https://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'  # Chrome 브라우저를 다운로드할 수 있도록 저장소 추가
!apt-get update  # 패키지 목록 다시 업데이트
!apt-get install -y google-chrome-stable  # 최신 안정 버전의 Chrome 브라우저 설치
!apt-get install -y fonts-nanum # 한글폰트 설치

## 라이브러리 import


In [2]:
# Selenium과 webdriver-manager를 사용한 Chrome 브라우저 실행 코드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4

# WebDriver 설치 및 브라우저 실행 설정
options = webdriver.ChromeOptions()
## 리눅스에서는 필수임!! CLI환경에선 그래픽을 지원하지않으므로
# options.add_argument("--headless")  # 헤드리스 모드로 실행 (브라우저 창을 표시하지 않음)
options.add_argument("window-size=1920x1080") # 헤드리스 모드로 실행 (명시적으로 표기)
# options.add_argument("window-size=700x1100")
options.add_argument("--no-sandbox")  # 샌드박스 모드 비활성화 (가상 환경에서 안정적인 실행을 위해)
options.add_argument("--disable-dev-shm-usage")  # /dev/shm 사용 비활성화 (메모리 부족 방지)
options.add_argument("--disable-gpu")  # GPU 가속 비활성화 (필요시 사용)
options.add_argument("start-maximized")  # 브라우저 창 최대화, 반응형웹 같은경우 특정 버튼이 비활성활 될수도 있음, 헤드리스에서는 안먹힌다.
options.add_argument("disable-infobars")  # 정보 표시줄 비활성화
options.add_argument("--disable-extensions")  # 브라우저 확장 프로그램 비활성화
# user-agent 값 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36" #일반 사용자인 척 한다.
options.add_argument(f"user-agent={user_agent}")

# WebDriver로 Chrome 브라우저 실행
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# 웹페이지 파싱 될때까지 최대 3초 기다림
# driver.implicitly_wait(3)

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd

## selenium으로 데이터 수집

# 서칭 -> selenium으로 동적으로 문서로드 -> bs4로 원하는데이터 추출 -> 저장

## 키워드로 검색
: 키워드를 통해 네이버 플레이스 id 및 가맹점 정보 저장하기

In [1]:
def handle_error(error):
    """
    예외 처리 함수: 발생한 오류의 위치(함수 이름, 라인)와 내용을 출력합니다.

    Args:
    - error (Exception): 처리할 예외 객체

    """
    tb = traceback.extract_tb(error.__traceback__)
    fileName, lineNo, functionName, text = tb[-1]
    print(f"ErrorLocation : Function Name: {functionName} Line : {lineNo}")
    print(f"Error : {error}\n")

In [3]:
import requests,bs4
import pandas as pd

In [45]:
from enum import Enum, auto

In [2]:
import traceback

In [72]:
def get_store_url(keyword,sortVal='relative'):
    sort = {"relative":0,"distance":1} #0:관련도순 , 1:거리순    
    searchUrl = "https://m.map.naver.com//search2/search.naver?query={keyword}&siteSort={sort}"\
                .format(keywordss=keyword, sort=sort[sortVal])
    return searchUrl


In [73]:
try:
    get_store_url("화곡역 맛집","distance")    
except Exception as e:
    handle_error(e)

ErrorLocation : Function Name: get_store_url Line : 3
Error : 'keyword'



In [2]:
# 필요 라이브러리 정의
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4
import pandas as pd
from sqlalchemy import create_engine, text
import traceback

#에러 handler
def handle_error(error):
    """
    예외 처리 함수: 발생한 오류의 위치(함수 이름, 라인)와 내용을 출력합니다.

    Args:
    - error (Exception): 처리할 예외 객체

    """
    tb = traceback.extract_tb(error.__traceback__)
    fileName, lineNo, functionName, text = tb[-1]
    print(f"ErrorLocation : Function Name: {functionName} Line : {lineNo}")
    print(f"Error : {error}\n")
# 데이터베이스 엔진 생성 함수
def createDB_Engine(db_info):
    engine = create_engine(f"{db_info['dbPrefix']}://{db_info['dbId']}:{db_info['dbPw']}@{db_info['dbIp']}:{db_info['dbPort']}/{db_info['dbName']}")
    return engine

# 데이터베이스에 테이블 생성
def create_table(engine,query):
    with engine.connect() as connection:
        connection.execute(text(query))
    print("테이블이 생성되었습니다.")

# 웹드라이버 셋업 함수
def setup_webdriver(osType='linux'):
    options = webdriver.ChromeOptions()
    if osType == 'linux':
        options.add_argument("--headless")  # 헤드리스 모드
    options.add_argument("window-size=1920x1080")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    
    # User-agent 설정
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# 프레임간 포커스 이동
def switch_frame(frameName):
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)

# 페이지 로딩 및 HTML 소스 반환 함수
def load_page(driver, url):
    
    driver.get(url)
    driver.implicitly_wait(10)
    time.sleep(3)  # 동적으로 생성되는 요소를 위한 추가 대기
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    return driver.page_source

# 가게 정보 파싱 함수
def parse_store_data(pageSource):
    bsObj = bs4.BeautifulSoup(pageSource, 'html.parser')
    storeListTags = bsObj.find_all(name="li", attrs={"class": "_item _lazyImgContainer"})
    storeList = []
    
    for storeTag in storeListTags:
        placeId = storeTag.get("data-id") if storeTag.get("data-sid") is None else storeTag.get("data-sid")
        storeName = storeTag.get("data-title",None)
        storeTel = storeTag.get("data-tel",None)
        storeLat = storeTag.get("data-latitude",None)
        storeLon = storeTag.get("data-longitude",None)
        storeInfo = {
            "place_id": placeId,
            "name": storeName,
            "tel": storeTel,
            "latitude": storeLat,
            "longitude": storeLon,
            "gibun" : None,
            "addr" : None
        }

        # 주소 정보 파싱
        addressDivTag = storeTag.find(name="div", attrs={"class": "bx_address"})
        
        for addr in addressDivTag.find_all('p'):
            tempAddr = addr.get_text(strip=True)
            if '지번' in tempAddr:
                #주소가 없을경우
                storeInfo['gibun'] = tempAddr[2:]  # '지번' 이후 주소
            else:
                storeInfo['addr'] = tempAddr
                
        storeList.append(storeInfo)

    # 데이터프레임으로 변환
    df = pd.DataFrame(storeList)
    return df

# 데이터베이스에 데이터 저장 함수
def save_data(engine,df, tableName):
    try:
        # 데이터프레임을 SQL 테이블로 저장 (중복된 placeId가 있는 경우 업데이트)
         with engine.begin() as connection: #engin.begin 자동 트랜잭션
            totalRow = 0
            for index, row in df.iterrows():
                insert_query = f"""
                INSERT INTO {tableName} (place_id, name, tel, latitude, longitude, addr, gibun, search_keyword, search_sort_type)
                VALUES (:place_id, :name, :tel, :latitude, :longitude, :addr, :gibun, :search_keyword, :search_sort_type)
                ON DUPLICATE KEY UPDATE
                    name = VALUES(name),
                    tel = VALUES(tel),
                    latitude = VALUES(latitude),
                    longitude = VALUES(longitude),
                    addr = VALUES(addr),
                    gibun = VALUES(gibun),
                    search_keyword = VALUES(search_keyword),
                    search_sort_type = VALUES(search_sort_type);
                """
                try:
                    result = connection.execute(text(insert_query), row.to_dict())
                    totalRow += 1
                    # logging.info(f"Row {index}: Affected rows for place_id {row['place_id']}: {result.rowcount}")
                except Exception as e:
                    logging.error(f"Error inserting row {index}: {str(e)}")
                    logging.error(f"Problematic row data: {row.to_dict()}")
            print(f"테이블 '{tableName}' 에 총 row '{totalRow}' 데이터가 성공적으로 삽입되었습니다.")
    except Exception as e:
        print(f"테이블에 데이터 삽입 중 오류 발생: {str(e)}")

# 가게 URL 생성 함수
def get_store_url(keyword, sortVal):
    sort = {"relative": 0, "distance": 1}  # 0: 관련도순, 1: 거리순
    searchUrl = f"https://m.map.naver.com/search2/search.naver?query={keyword}&siteSort={sort[sortVal]}"
    return searchUrl

# 메인 실행 부분
try:
    osType = 'mac'
    driver = setup_webdriver(osType)
        # 데이터베이스 정보 설정
    dbInfo = {
        "dbPrefix": "mysql+pymysql",
        "dbId": "crawling",
        "dbPw": "cr1234",
        "dbIp": "54.180.233.91",
        "dbPort": "3306",
        "dbName": "crawling",
    }
    tableName = "store_info"
    createTablequery = """
        CREATE TABLE IF NOT EXISTS {} (
        place_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255),
        tel VARCHAR(255),
        latitude VARCHAR(255),
        longitude VARCHAR(255),
        addr VARCHAR(255),
        gibun VARCHAR(255),
        search_keyword VARCHAR(255),
        search_sort_type VARCHAR(255),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,  -- 기본값으로 현재 시간 설정
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP  -- 업데이트 시 현재 시간으로 자동 설정
        );
    """.format(tableName)
    {}
    # 검색어와 정렬 조건을 사용해 URL 생성
    keyword = "화곡역 맛집"
    sortType = "distance"
    searchUrl = get_store_url(keyword, sortType)
    
    # 페이지 로드 및 데이터 추출
    html = load_page(driver, searchUrl)
    
    df = parse_store_data(html)
    df['search_keyword'] = keyword #서칭 키워드 컬럼 추가
    df['search_sort_type'] = sortType #서칭 정렬 컬럼 추가
    
    
    # 데이터베이스 엔진 생성 및 테이블 생성
    # engine = createDB_Engine(dbInfo)
    # create_table(engine,createTablequery)
    
    # 추출한 데이터 저장
    # save_data(engine,df, tableName)

except Exception as e:
    handle_error(e)
    
finally:
    driver.quit()  # 드라이버 종료


In [ ]:
createKeywordTableQuery = """
        CREATE TABLE IF NOT EXISTS keyword (
        idx INT AUTO_INCREMENT PRIMARY KEY,
        keyword VARCHAR(255) NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP  -- 기본값으로 현재 시간 설정
        );
    """
createKeywordPlaceRelationTableQuery = """
        CREATE TABLE IF NOT EXISTS keyword_place_relation (
        idx INT AUTO_INCREMENT PRIMARY KEY,
        keyword_idx INT NOT NULL,
        place_id VARCHAR(255) NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,  -- 기본값으로 현재 시간 설정
        FOREIGN KEY (keyword_idx) REFERENCES keyword(idx) ON DELETE CASCADE  -- 키워드 테이블과 관계 설정
        );
    """
createTablequery = """
    CREATE TABLE IF NOT EXISTS {} (
    place_id VARCHAR(255) PRIMARY KEY,
    name VARCHAR(255),
    tel VARCHAR(255),
    latitude VARCHAR(255),
    longitude VARCHAR(255),
    addr VARCHAR(255),
    gibun VARCHAR(255),
    search_keyword VARCHAR(255),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,  -- 기본값으로 현재 시간 설정
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP  -- 업데이트 시 현재 시간으로 자동 설정
    );
""".format(tableName)

engine = createDB_Engine(dbInfo)
create_table(engine,createTablequery)



In [174]:
# 필요 라이브러리 정의
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4
import pandas as pd
from sqlalchemy import create_engine, text
import traceback

#에러 handler
def handle_error(error):
    """
    예외 처리 함수: 발생한 오류의 위치(함수 이름, 라인)와 내용을 출력합니다.

    Args:
    - error (Exception): 처리할 예외 객체

    """
    tb = traceback.extract_tb(error.__traceback__)
    fileName, lineNo, functionName, text = tb[-1]
    print(f"ErrorLocation : Function Name: {functionName} Line : {lineNo}")
    print(f"Error : {error}\n")

# 웹드라이버 셋업 함수
def setup_webdriver(osType='linux'):
    options = webdriver.ChromeOptions()
    if osType == 'linux':
        options.add_argument("--headless")  # 헤드리스 모드
    options.add_argument("window-size=1920x1080")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-images")  # 이미지 로딩 비활성화
    options.add_argument("--disable-gpu")
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    
    # User-agent 설정
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver
## Selenium 함수 ##

def scroll_to_bottom(driver, scrollableElement):
    loadPreHeight = driver.execute_script("return arguments[0].scrollHeight", scrollableElement)
    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 600;", scrollableElement)
        # 페이지 로드를 기다림
        time.sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절
        # 새 높이 계산
        loadAftHeight = driver.execute_script("return arguments[0].scrollHeight", scrollableElement)
        if loadAftHeight == loadPreHeight:
            break
        loadPreHeight = loadAftHeight  
    return True

def is_last_page(driver,nextPageTag="div.zRM9F > a:last-child"):
    not_next_page = driver.find_element(By.CSS_SELECTOR, nextPageTag).get_attribute('aria-disabled')
    return not_next_page == 'true'
    
# 프레임간 포커스 이동
def forcus_switch_frame(driver,frameSelector):
    # "#searchIframe" 상점 이름
    # "#entryIframe" 상점 정보
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.CSS_SELECTOR,'{}'.format(frameSelector))
    driver.switch_to.frame(iframe)
    return driver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep

# 사용 예시
# from selenium import webdriver
# driver = webdriver.Chrome()
# store_data = scrape_store(driver, "https://example.com/store/page")
# print(store_data)
# driver.quit()

class CustomError(Exception):
    def __init__(self, message, data):
        super().__init__(message)
        self.data = data
    
def load_page(driver, url,storeList):
    #변수
    result = False
    isLastPage = False
    storeList = storeList
    
    
  # 대기 시간
    driver.get(url)
    driver.implicitly_wait(10)
    time.sleep(3)  # 동적으로 생성되는 요소를 위한 추가 대기
    
    try:    
        while(True):
            pageStoreList = []
            driver = forcus_switch_frame(driver,"#searchIframe")
            
            # 페이지 숫자를 초기에 체크 [ True / False ]
            # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
            # 다음 페이지 요소
            nextPageTag = "div.zRM9F > a:last-child"
            
            # notNextPage = driver.find_element(By.CSS_SELECTOR,nextPageTag).get_attribute('aria-disabled')
            # if(notNextPage == 'true'):
            #     isLastPage = True
                
            currPage = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text    
            if(len(storeList) >= int(currPage)):
                driver.find_element(By.CSS_SELECTOR,nextPageTag).click()
                time.sleep(2)
                continue
            else:
                print("현재페이지",currPage)
                
            ############## 맨 밑까지 스크롤 ##############
            scrollableElement = driver.find_element(By.CLASS_NAME, "Ryr1F")
            resultScroll = scroll_to_bottom(driver, scrollableElement)
            
            if(resultScroll):
                print('스크롤 완료')
            
            # 현재 페이지에 등록된 모든 가게 조회
            storeListTags = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')    
         
            for index, storeTag in enumerate(storeListTags):
            # 광고 태그 찾기 
                try:
                    #광고 태그있을때
                    storeTag.find_element(By.CSS_SELECTOR, 'a.gU6bV[target="_blank"][role="button"]')
                    continue
                except:
                    pass
                    #광고 태그없을때
                
                #한사이클
                try:
                    storeName = None # 가게 이름
                    category = None # 카테고리
                    # rating = 0.0 # 전체 평점
                    visitedReviewCnt = 0 # 방문자 리뷰
                    blogReviewCnt = 0 # 블로그 리뷰
                    placeId = None # 가게 고유 번호
                    addr = None # 가게 주소
                    businessHours = '' # 영업 시간
                    storeTel = None # 전화번호
                    
                    # 순서대로 값을 하나씩 클릭
                
                    driver =  forcus_switch_frame(driver,"#searchIframe") #click때문에 프레임이동
                
                
                    storeTag.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()
                                # e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()
                    time.sleep(2)
        
                    # entryIframe에서 정보 가져오기
                    driver =  forcus_switch_frame(driver,"#entryIframe")
                    
                    #가게이름,카테고리,방문자 리뷰 갯수,블로그 리뷰 갯수
                    header = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
                    
                    # 가게 이름
                    storeName = header.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
                    # 카테고리
                    category = header.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text
        
                    try:
                        #최대 3개 평점,방문자,블로그
                        reviewTagLen = len(header.find_elements(By.XPATH,'.//div[2]/span'))
                        # 방문자 리뷰   
                        visitedReviewCnt = header.find_element(By.XPATH,f'.//div[2]/span[{reviewTagLen-1}]/a').text
                        # 블로그 리뷰
                        blogReviewCnt = header.find_element(By.XPATH,f'.//div[2]/span[{reviewTagLen}]/a').text
    
                    except Exception as e:
                        print(e)
                        print(storeName+"리뷰를 가져오는데 실패했습니다.")
    
                    try:
                    
                        # 전화번호 요소 기다림
                        storeTelElem = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '//span[@class="xlx7Q"]'))
                        )
                        
                        # 전화번호 요소가 화면에 나타나도록 스크롤
                        driver.execute_script("arguments[0].scrollIntoView();", storeTelElem)
                    
                        # 요소가 가려져 있지 않도록, 다시 기다림
                        WebDriverWait(driver, 10).until(
                            EC.visibility_of(storeTelElem)
                        )
                        storeTel = storeTelElem.text

                    
                    except Exception as e:
                        print(f"{storeName} 전화번호를 가져오는데 실패했습니다.")
    
                    
                    # 가게 id
            
                    placeId = driver.find_element(By.XPATH,'//div[@class="flicking-camera"]/a').get_attribute('href').split('/')[4]
                  
                    # 가게 주소
                   
                    addr = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text
                   
                    
                    storeInfo = {
                        "place_id": placeId,
                        "name": storeName,
                        "tel": storeTel,
                        "visited_review_cnt" : visitedReviewCnt,
                        "blog_review_cnt" : blogReviewCnt,
                        "business_hours" : businessHours,
                        "addr" : addr
                    }
                    print("storeInfo")
                    pageStoreList.append(storeInfo)

                #invalid Session id 처리 하기
                except Exception as e:
                    if "invalid session id" in str(e):
                        print('재시작 필요')
                        return False, storeList 
                    else:
                        continue

            storeList.append(pageStoreList)
            print('상점정보 크롤링 완료')
    
            driver =  forcus_switch_frame(driver,"#searchIframe")
            
            #마지막 페이지확인
            if is_last_page(driver):
                print("마지막")
                break
            else:
                print("다음없어")
                #다음페이지로 이동
                driver.find_element(By.CSS_SELECTOR,nextPageTag).click()
                #app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7)
        
        result = True
        return result,storeList
    except Exception as e:
            # 특정 오류 처리: invalid session id
            if "invalid session id" in str(e):
                print('재시작 필요')
                return False, storeList 
            else:
                raise CustomError(e, storeList)  # 예외와 데이터를 함께 전달

# 가게 정보 파싱 함수

##데이터 베이스 관련 함수##
# 데이터베이스 엔진 생성 함수
def createDB_Engine(db_info):
    engine = create_engine(f"{db_info['dbPrefix']}://{db_info['dbId']}:{db_info['dbPw']}@{db_info['dbIp']}:{db_info['dbPort']}/{db_info['dbName']}")
    return engine

# 데이터베이스에 테이블 생성
def create_table(engine,query):
    with engine.connect() as connection:
        connection.execute(text(query))
    print("테이블이 생성되었습니다.")

# 데이터베이스에 데이터 저장 함수
def save_data(engine,df, tableName):
    try:
        # 데이터프레임을 SQL 테이블로 저장 (중복된 placeId가 있는 경우 업데이트)
         with engine.begin() as connection: #engin.begin 자동 트랜잭션
            totalRow = 0
            for index, row in df.iterrows():
                insert_query = f"""
                INSERT INTO {tableName} (place_id, name, tel, latitude, longitude, addr, gibun, search_keyword, search_sort_type)
                VALUES (:place_id, :name, :tel, :latitude, :longitude, :addr, :gibun, :search_keyword, :search_sort_type)
                ON DUPLICATE KEY UPDATE
                    name = VALUES(name),
                    tel = VALUES(tel),
                    latitude = VALUES(latitude),
                    longitude = VALUES(longitude),
                    addr = VALUES(addr),
                    gibun = VALUES(gibun),
                    search_keyword = VALUES(search_keyword),
                    search_sort_type = VALUES(search_sort_type);
                """
                try:
                    result = connection.execute(text(insert_query), row.to_dict())
                    totalRow += 1
                    # logging.info(f"Row {index}: Affected rows for place_id {row['place_id']}: {result.rowcount}")
                except Exception as e:
                    logging.error(f"Error inserting row {index}: {str(e)}")
                    logging.error(f"Problematic row data: {row.to_dict()}")
            print(f"테이블 '{tableName}' 에 총 row '{totalRow}' 데이터가 성공적으로 삽입되었습니다.")
    except Exception as e:
        print(f"테이블에 데이터 삽입 중 오류 발생: {str(e)}")


# 메인 실행 부분
try:
    #driver 생성
    osType = 'linux'
  
    # 검색어와 정렬 조건을 사용해 URL 생성
    keyword = "화곡역 맛집"
    searchUrl = get_store_url(keyword)
    # 페이지 로드 및 데이터 추출

    result = False
    storeList = []
    #중간에 세션문제 때문에 저장한페이지 이후부터 시작
    start_time = time.time()
    
    while not result:
        try:
          driver = setup_webdriver(osType)
          result,storeList = load_page(driver, searchUrl,storeList)
          driver.quit()
        except CustomError as e:
            print(f"에러 메시지: {e}")
            storeList = e.data
            break  # 일반적인 예외 발생 시 루프 종료
        except Exception as e:
            break  # 일반적인 예외 발생 시 루프 종료
        finally:
            driver.quit()  # driver 종료   
    end_time = time.time()
    # 실행 시간 출력 (초 단위)
    print(f"실행 시간: {end_time - start_time} 초")
    
    ##DB

    # df = parse_store_data(html)
    # df['search_keyword'] = keyword #서칭 키워드 컬럼 추가
    # df['search_sort_type'] = sortType #서칭 정렬 컬럼 추가
    
    
    # 데이터베이스 엔진 생성 및 테이블 생성
    # engine = createDB_Engine(dbInfo)
    # create_table(engine,createTablequery)
    
    # 추출한 데이터 저장
    # save_data(engine,df, tableName)

except Exception as e:
    handle_error(e)
    
finally:
    pass
    driver.quit()  # 드라이버 종료


현재페이지 1
스크롤 완료
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
storeInfo
Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[2]/span[0]/a"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104614274 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x000000010460c75c cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000104220260 cxxbridge1$string$len + 89488
3   chromedriver                        0x000000010426450c cxxbridge1$string$len + 368700
4   chromedriver                        0x000000010425aad0 cxxbridge1$string$len + 329216
5   chromedriver                        0x000000010429e7d0 cxxbridge1$string$len + 606976

In [175]:
df = pd.DataFrame(storeList[0])
for i in range(5):
    print(len(storeList[i]))

74
74
70
70
20


In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import random
import re

from selenium import webdriver
import sys

class Colors:
    # 텍스트 색상
    RESET = '\033[0m'       # 모든 설정 초기화
    RED = '\033[91m'        # 빨간색 텍스트
    GREEN = '\033[92m'      # 초록색 텍스트
    YELLOW = '\033[93m'     # 노란색 텍스트
    BLUE = '\033[94m'       # 파란색 텍스트
    MAGENTA = '\033[95m'    # 보라색 텍스트
    CYAN = '\033[96m'       # 청록색 텍스트
    WHITE = '\033[97m'      # 흰색 텍스트
    
    # 배경색
    BG_RED = '\033[41m'     # 빨간색 배경
    BG_GREEN = '\033[42m'   # 초록색 배경
    BG_YELLOW = '\033[43m'  # 노란색 배경
    BG_BLUE = '\033[44m'    # 파란색 배경
    BG_MAGENTA = '\033[45m' # 보라색 배경
    BG_CYAN = '\033[46m'    # 청록색 배경
    BG_WHITE = '\033[47m'   # 흰색 배경
    
    # 텍스트 스타일
    BOLD = '\033[1m'        # 굵게
    UNDERLINE = '\033[4m'   # 밑줄


def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.CSS_SELECTOR,'{}'.format("#searchIframe"))
    driver.switch_to.frame(iframe)
    
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id=""]')
    driver.switch_to.frame(iframe)

# 포커스 변경
def forcus_switch_frame(driver,frameSelector):
    # "#searchIframe" 상점 이름
    # "#entryIframe" 상점 정보
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.CSS_SELECTOR,'{}'.format(frameSelector))
    driver.switch_to.frame(iframe)
    return driver

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

# 대기 시간
driver.implicitly_wait(time_to_wait=3)

# 반복 종료 조건
loop = True

URL = 'https://map.naver.com/p/search/%EC%9D%B8%ED%95%98%EB%8C%80%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=15.23,0,0,0,dh'
driver.get(url=URL)

while(True):
    switch_left()

    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[7]').get_attribute('aria-disabled')
    if(next_page == 'true'):
        break

    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)

        # 페이지 로드를 기다림
        sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절

        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        print("새로운",new_height)
        print("기존",last_height)
        if new_height == last_height:
            break

        last_height = new_height


    ############## 현재 page number 가져오기 - 1 페이지 ##############

    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text

    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함
    if(page_no == '1'):
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')[2:]
    else:
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')

    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elemets)) + '\033[0m' + '개의 가게를 찾았습니다.\n')

    for index, e in enumerate(elemets, start=1):
        final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span")

        print(str(index) + ". " + final_element.text)

    print(Colors.RED + "-"*50 + Colors.RESET)

    switch_left()

    sleep(2)

    for index, e in enumerate(elemets, start=1):
        store_name = '' # 가게 이름
        category = '' # 카테고리
        new_open = '' # 새로 오픈
        
        rating = 0.0 # 평점
        visited_review = 0 # 방문자 리뷰
        blog_review = 0 # 블로그 리뷰
        store_id = '' # 가게 고유 번호
        
        address = '' # 가게 주소
        business_hours = [] # 영업 시간
        phone_num = '' # 전화번호

        switch_left()


        # 순서대로 값을 하나씩 클릭
        e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()

        sleep(2)

        switch_right()

        ################### 여기부터 크롤링 시작 ##################
        title = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
        store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')


        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text

        # 카테고리
        category = title.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text

        if(len(store_info) > 2):
            # 새로 오픈
            new_open = title.find_element(By.XPATH,'.//div[1]/div[1]/span[3]').text


        ###############################

        review = title.find_elements(By.XPATH,'.//div[2]/span')

        # 인덱스 변수 값
        _index = 1

        # 리뷰 ROW의 갯수가 3개 이상일 경우 [별점, 방문자 리뷰, 블로그 리뷰]
        if len(review) > 2:
            rating_xpath = f'.//div[2]/span[{_index}]'
            rating_element = title.find_element(By.XPATH, rating_xpath)
            rating = rating_element.text.replace("\n", " ")

            _index += 1

        try:
            # 방문자 리뷰
            visited_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text


            # 인덱스를 다시 +1 증가 시킴
            _index += 1

            # 블로그 리뷰
            blog_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text
        except:
            print(Colors.RED + '------------ 리뷰 부분 오류 ------------' + Colors.RESET)

        # 가게 id
        store_id = driver.find_element(By.XPATH,'//div[@class="flicking-camera"]/a').get_attribute('href').split('/')[4]

        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text


        try:
            driver.find_element(By.XPATH,'//div[@class="y6tNq"]//span').click()

            # 영업 시간 더보기 버튼을 누르고 2초 반영시간 기다림
            sleep(2)


            parent_element = driver.find_element(By.XPATH,'//a[@class="gKP9i RMgN0"]')
            child_elements = parent_element.find_elements(By.XPATH, './*[@class="w9QyJ" or @class="w9QyJ undefined"]')

            for child in child_elements:
                # 각 자식 요소 내에서 클래스가 'A_cdD'인 span 요소 찾기
                span_elements = child.find_elements(By.XPATH, './/span[@class="A_cdD"]')

                # 찾은 span 요소들의 텍스트 출력
                for span in span_elements:
                    business_hours.append(span)
            
            # 가게 전화번호
            phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text

        except:
            print(print(Colors.RED + '------------ 영업시간 / 전화번호 부분 오류 ------------' + Colors.RESET))
        
        print(Colors.BLUE + f'{index}. ' + str(store_name) + Colors.RESET + ' · ' + str(category) + Colors.RED + str(new_open) + Colors.RESET)
        print('평점 ' + Colors.RED + str(rating) + Colors.RESET + ' / ' + visited_review + ' · ' + blog_review)
        print(f'가게 고유 번호 -> {store_id}')
        print('가게 주소 ' + Colors.GREEN + str(address) + Colors.RESET)
        print(Colors.CYAN + '가게 영업 시간' + Colors.RESET)
        for i in business_hours:
            print(i.text)
            print('')
        print('가게 번호 ' + Colors.GREEN + phone_num + Colors.RESET)
        print(Colors.MAGENTA + "-"*50 + Colors.RESET)
    
    switch_left()
        
    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page == 'false'):
        driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').click()
    # 아닐 경우 루프 정지
    else:
        loop = False

새로운 4064
기존 4025
새로운 4181
기존 4064
새로운 4220
기존 4181
새로운 6743
기존 4220
새로운 6821
기존 6743
새로운 6899
기존 6821
새로운 6977
기존 6899
새로운 7055
기존 6977
새로운 9393
기존 7055
새로운 9471
기존 9393
새로운 9549
기존 9471
새로운 9627
기존 9549
새로운 9705
기존 9627
새로운 12168
기존 9705
새로운 12246
기존 12168
새로운 12324
기존 12246
새로운 12402
기존 12324
새로운 12480
기존 12402
새로운 14887
기존 12480
새로운 14965
기존 14887
새로운 15043
기존 14965
새로운 15121
기존 15043
새로운 15199
기존 15121
새로운 17656
기존 15199
새로운 17734
기존 17656
새로운 17812
기존 17734
새로운 17890
기존 17812
새로운 17968
기존 17890
새로운 20431
기존 17968
새로운 20509
기존 20431
새로운 20587
기존 20509
새로운 20665
기존 20587
새로운 20743
기존 20665
새로운 22149
기존 20743
새로운 22188
기존 22149
새로운 22305
기존 22188
새로운 22305
기존 22305
현재 1 페이지 / 총 74개의 가게를 찾았습니다.

1. 백소정 인하대후문점
2. 만복국수집 인하대역점
3. 킹콩스테이크
4. 팔각도 인하대역점
5. 면식당 인하대점
6. 동추원불고기
7. 계절포차
8. 계속 쪼르면
9. 이모떡즉석떡볶이
10. 보화장
11. 숯불구이먹방
12. 0194아카이브
13. 미로곱창볶음집 인하대점
14. 카페삼층
15. 국제식당
16. 박지혜 모던순대국 소머리국밥 인하직영점
17. 금산양꼬치 본점
18. 카페 오디디오 본점
19. 인하칼국수 인하대후문점
20. 천지인
21. 넋넋
22. 덮수룩
23. 남도뽀글이 인하대본점
24. 서가앤쿡 인천 인

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id=""]"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001029b8274 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001029b075c cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x00000001025c4260 cxxbridge1$string$len + 89488
3   chromedriver                        0x000000010260850c cxxbridge1$string$len + 368700
4   chromedriver                        0x00000001026427d0 cxxbridge1$string$len + 606976
5   chromedriver                        0x00000001025fd12c cxxbridge1$string$len + 322652
6   chromedriver                        0x00000001025fdd7c cxxbridge1$string$len + 325804
7   chromedriver                        0x0000000102980504

In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import threading

# FastAPI 애플리케이션 인스턴스 생성
app = FastAPI()
class SearchQuery(BaseModel):
    keyword: str
    sort_type: str
    
@app.get("/")
async def root():
    return {"message": "Hello from FastAPI!"}


@app.post("/search")
async def search(query: SearchQuery):
    try:
        osType = 'linux'
        driver = setup_webdriver(osType)
            # 데이터베이스 정보 설정
        dbInfo = {
            "dbPrefix": "mysql+pymysql",
            "dbId": "crawling",
            "dbPw": "cr1234",
            "dbIp": "54.180.233.91",
            "dbPort": "3306",
            "dbName": "crawling",
        }
        tableName = "store_info"
        createTablequery = """
            CREATE TABLE IF NOT EXISTS {} (
            place_id VARCHAR(255) PRIMARY KEY,
            name VARCHAR(255),
            tel VARCHAR(255),
            latitude VARCHAR(255),
            longitude VARCHAR(255),
            addr VARCHAR(255),
            gibun VARCHAR(255),
            search_keyword VARCHAR(255),
            search_sort_type VARCHAR(255),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,  -- 기본값으로 현재 시간 설정
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP  -- 업데이트 시 현재 시간으로 자동 설정
            );
        """.format(tableName)
        
        # 검색어와 정렬 조건을 사용해 URL 생성
        keyword = query.keyword
        sortType = query.sort_type
        searchUrl = get_store_url(keyword, sortType)
        
        # 페이지 로드 및 데이터 추출
        html = load_page(driver, searchUrl)
        
        df = parse_store_data(html)
        df['search_keyword'] = keyword #서칭 키워드 컬럼 추가
        df['search_sort_type'] = sortType #서칭 정렬 컬럼 추가
        
        
        # 데이터베이스 엔진 생성 및 테이블 생성
        engine = createDB_Engine(dbInfo)
        create_table(engine,createTablequery)
        
        # 추출한 데이터 저장
        save_data(engine,df, tableName)

        return {"status": "success", "message": "Data saved successfully!"}

    except Exception as e:
        handle_error(e)
        raise HTTPException(status_code=500, detail=str(e))

    finally:
        driver.quit()

import nest_asyncio

# 이미 실행 중인 이벤트 루프에 중첩 적용
nest_asyncio.apply()


# Uvicorn 서버를 Jupyter Notebook의 이벤트 루프 내에서 실행
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=9393)



INFO:     Started server process [25935]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9393 (Press CTRL+C to quit)


INFO:     127.0.0.1:53851 - "GET / HTTP/1.1" 200 OK
테이블이 생성되었습니다.
테이블 'store_info' 에 총 row '75' 데이터가 성공적으로 삽입되었습니다.
INFO:     127.0.0.1:53855 - "POST /search HTTP/1.1" 200 OK
테이블이 생성되었습니다.
테이블 'store_info' 에 총 row '75' 데이터가 성공적으로 삽입되었습니다.
INFO:     127.0.0.1:53896 - "POST /search HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25935]


In [24]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/Users/sangmoonjeon/venv/github/PythonProject/Crawling']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [25408] using StatReload
ERROR:    Error loading ASGI app. Could not import module "main".
^C
INFO:     Stopping reloader process [25408]


### 참고)페이지가 동적으로 생성되어 단순 request로는 불가능

In [6]:

# html = requests.get(keywordUrl)
# html.encoding = 'utf-8'
# html = html.text
# bsObj = bs4.BeautifulSoup(markup=html, features = 'html.parser')
# print(bsObj.find(name="ul",attrs={"class":"search_list _items"}))

<ul class="search_list _items">
<!-- 업체 출력 List (search.tpl.xml[js]로 이동)-->
</ul>


## 리뷰 수집

In [82]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from math import ceil

def handle_error(error):
    """
    예외 처리 함수: 발생한 오류의 위치(함수 이름, 라인)와 내용을 출력합니다.

    Args:
    - error (Exception): 처리할 예외 객체

    """
    tb = traceback.extract_tb(error.__traceback__)
    fileName, lineNo, functionName, text = tb[-1]
    print(f"ErrorLocation : Function Name: {functionName} Line : {lineNo}")
    print(f"Error : {error}\n")


def get_review_url(placeId):
    reviewUrl = f"https://pcmap.place.naver.com/restaurant/{placeId}/review/visitor?reviewSort=recent"
    return reviewUrl
    
def setup_webdriver(osType='linux'):
    options = webdriver.ChromeOptions()
    if osType == 'linux':  # 문자열로 비교
        options.add_argument("--headless")  # 헤드리스 모드
    options.add_argument("window-size=1920x1080")
    options.add_argument("--no-sandbox")  
    options.add_argument("--disable-dev-shm-usage")  
    options.add_argument("--disable-gpu")  
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")  
    options.add_argument("--disable-extensions")
    
    # User-agent 설정
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver
    
def load_page(driver, url, selectors):
    #더보기 버튼과 리뷰 카운트 셀렉터 가져오기
    addBtnSel = selectors["addBtnSel"]
    reviewCntSel = selectors["reviewCntSel"]
    
    driver.get(url)
    driver.implicitly_wait(10)

    # 첫 번째 버튼 찾기 및 클릭
    wait = WebDriverWait(driver, 10)  # 10초 동안 대기
    
    # 리뷰 총 갯수 찾기
    # element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, reviewCntSel)))
    element = driver.find_element(By.CSS_SELECTOR, reviewCntSel)
    totalReviewCnt = int(element.text)
    RENDER_TIME = selectors["RENDER_TIME"]

    #기존 10개 제외하고 10으로 나누기
    finalTry = ceil((totalReviewCnt - 10) / 10)
    
    for j in range(finalTry):
        try:
            time.sleep(RENDER_TIME)  # 동적요소 랜더링 대기
            # driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            name = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, addBtnSel))
            )
            # 특정 요소로 스크롤
            driver.execute_script("arguments[0].scrollIntoView();", name)
            driver.execute_script("arguments[0].click();", name)

            # 마지막 렌더링 대기
            if (finalTry-1 == j):
                time.sleep(RENDER_TIME)
                
        except Exception as e:
            print(e)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            break
    
    return driver.page_source  # HTML 소스 반환

def parse_review_data(html, selectors):
    bsObj = bs4.BeautifulSoup(html, "html.parser")
    
    # selectors 딕셔너리에서 선택자 가져오기
    reviewListSel = selectors["reviewListSel"] 
    reviewIdSel = selectors["reviewIdSel"]
    reviewContentSel = selectors["reviewContentSel"]
    reviewDateTag = selectors["reviewDateTag"]

    # CSS selector로 리뷰 리스트 찾기
    reviewLists = bsObj.select(reviewListSel)

    columnList = ["리뷰작성ID", "리뷰내용", "리뷰일시"]
    rowList = []

    for review in reviewLists:
        # CSS selector로 각각의 리뷰 아이디와 내용을 찾기
        reviewId = review.select_one(reviewIdSel).text
        reviewContent = review.select_one(reviewContentSel).text
        reviewDate = review.find(reviewDateTag).text  # 시간 요소는 그대로 find 사용

        rowList.append([reviewId, reviewContent, reviewDate])

    df = pd.DataFrame(rowList, columns=columnList)
    return df
    

try:
    placeId = '1991254323'
    driver = setup_webdriver('window')
    
    #selectors 딕셔너리
    selectors = {
        "RENDER_TIME":3,
        "addBtnSel": 'a.fvwqf',  # 더보기 버튼
        "reviewCntSel": 'em.place_section_count',  # 리뷰 총 갯수
        "reviewContentSel": 'div.pui__vn15t2',  # 리뷰 내용
        "reviewIdSel": "span.pui__NMi-Dp",  # 리뷰 작성 id
        "reviewListSel": "li.pui__X35jYm",  # 리뷰 리스트 아이템
        "reviewDateTag": "time"  # 리뷰 날짜 태그
    }
    #store 정보 로드
    searchUrl = get_review_url(placeId)
    html = load_page(driver, searchUrl, selectors)
    df = parse_review_data(html, selectors)
    # 데이터 확인
except Exception as e:
    handle_error(e)
finally:
    driver.quit()  # 드라이버 종료 (예외가 발생해도 안전하게 종료)


Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.58)
Stacktrace:
	GetHandleVerifier [0x011B83E3+25571]
	(No symbol) [0x0114A684]
	(No symbol) [0x01042113]
	(No symbol) [0x0101E23B]
	(No symbol) [0x010B17FF]
	(No symbol) [0x010C4C99]
	(No symbol) [0x010AA9A6]
	(No symbol) [0x0107BAB6]
	(No symbol) [0x0107C50D]
	GetHandleVerifier [0x0148C4A3+2991267]
	GetHandleVerifier [0x014DD2C9+3322569]
	GetHandleVerifier [0x012484D2+615634]
	GetHandleVerifier [0x0124FBFC+646140]
	(No symbol) [0x0115327D]
	(No symbol) [0x01150188]
	(No symbol) [0x01150325]
	(No symbol) [0x01142826]
	BaseThreadInitThunk [0x76BFFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776380CE+286]
	RtlGetAppContainerNamedObjectPath [0x7763809E+238]



NameError: name 'traceback' is not defined

In [81]:
len(df)

464

In [ ]:
# placeId = "1194099490"
for i in range(0,len(storeList)):
    placeId = storeList[i]["placeId"]
    reviewUrl = "https://pcmap.place.naver.com/restaurant/{}/review/visitor".format(placeId)
    directoryPath = "./places"
    os.makedirs(directoryPath, exist_ok=True)
    
    #더보기 버튼
    addBtnSelector = 'a.fvwqf'
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # URL 이동
    driver.get(reviewUrl)
    
    # 웹페이지 파싱 될때까지 최대 3초 기다림
    driver.implicitly_wait(3)
    
    #더보기 xpath
    # xPath = '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div'
    # name = WebDriverWait(driver, 10).until(
    #                     EC.element_to_be_clickable(
    #                         (By.CSS_SELECTOR, addBtnSelector)
    #                     )
    #                 )
    # driver.execute_script("arguments[0].click();", name)
    # 첫 번째 버튼 찾기 및 클릭
    wait = WebDriverWait(driver, 10)  # 10초 동안 대기
    dfList = []
    storeName = storeList[i]['name']
    for j in range(0,10):
    # for _ in iter(int, 1):
        print(storeName)
        print(j,"  : 회차")
        try:
           #더보기 버튼
           # driver.find_element(By.XPATH, xPath).click()
           driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
           # addBtn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,addBtnSelector)))
           # addBtn.click()
           name = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable(
                            (By.CSS_SELECTOR, addBtnSelector)
                        )
                    )
           driver.execute_script("arguments[0].click();", name)
           #디버그
           # html = driver.page_source
           # dfList.append(debug_bsHtml(html))
        except Exception as e:
            print(e)
            print("없다")
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            #디버그
            #html = driver.page_source
            #dfList.append(debug_bsHtml(html))
            break
    
    html = driver.page_source
    bsHtml(html,directoryPath,placeId,storeName)

# BeautifulSoup로 페이지 소스 파싱
# bsObj = bs4.BeautifulSoup(html,"html.parser")

# reviewListCls = "pui__X35jYm EjjAW" 
# reviewIdCls = "pui__NMi-Dp" #span
# reviewContentCls = "pui__vn15t2" #div

# reviewLists = bsObj.findAll(name="li",
#                               attrs={"class":reviewListCls})

# columnList = ["리뷰작성ID","리뷰내용"]
# rowList = []
# eachRowList = []
# for i in range(len(reviewLists)):
#     reviewId = reviewLists[i].find(name="span",
#                    attrs={"class":reviewIdCls}).text
#     reviewContent = reviewLists[i].find(name="div",
#                        attrs={"class":reviewContentCls}).text
#     eachRowList.append(reviewId)
#     eachRowList.append(reviewContent)
#     rowList.append(eachRowList)
#     eachRowList = []

# df = pd.DataFrame(rowList,columns=columnList)
# #place별 저장
# placePath = path.join(directoryPath,placeId)
# os.makedirs(placePath, exist_ok=True)
# df.to_csv(path.join(placePath,"reviewData.csv"))    




In [20]:
# placeId = "1194099490"
for i in range(0,len(storeList)):
    placeId = storeList[i]["placeId"]
    reviewUrl = "https://pcmap.place.naver.com/restaurant/{}/review/visitor".format(placeId)
    directoryPath = "./places"
    os.makedirs(directoryPath, exist_ok=True)
    
    #더보기 버튼
    addBtnSelector = 'a.fvwqf'
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # URL 이동
    driver.get(reviewUrl)
    
    # 웹페이지 파싱 될때까지 최대 3초 기다림
    driver.implicitly_wait(3)
    
    #더보기 xpath
    # xPath = '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div'
    # name = WebDriverWait(driver, 10).until(
    #                     EC.element_to_be_clickable(
    #                         (By.CSS_SELECTOR, addBtnSelector)
    #                     )
    #                 )
    # driver.execute_script("arguments[0].click();", name)
    # 첫 번째 버튼 찾기 및 클릭
    wait = WebDriverWait(driver, 10)  # 10초 동안 대기
    dfList = []
    storeName = storeList[i]['name']
    for j in range(0,3):
    # for _ in iter(int, 1):
        print(storeName)
        print(j,"  : 회차")
        try:
           #더보기 버튼
           # driver.find_element(By.XPATH, xPath).click()
           driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
           # addBtn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,addBtnSelector)))
           # addBtn.click()
           name = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable(
                            (By.CSS_SELECTOR, addBtnSelector)
                        )
                    )
           driver.execute_script("arguments[0].click();", name)
           #디버그
           # html = driver.page_source
           # dfList.append(debug_bsHtml(html))
        except Exception as e:
            print(e)
            print("없다")
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            #디버그
            #html = driver.page_source
            #dfList.append(debug_bsHtml(html))
            break
    
    html = driver.page_source
    bsHtml(html,directoryPath,placeId,storeName)

# BeautifulSoup로 페이지 소스 파싱
# bsObj = bs4.BeautifulSoup(html,"html.parser")

# reviewListCls = "pui__X35jYm EjjAW" 
# reviewIdCls = "pui__NMi-Dp" #span
# reviewContentCls = "pui__vn15t2" #div

# reviewLists = bsObj.findAll(name="li",
#                               attrs={"class":reviewListCls})

# columnList = ["리뷰작성ID","리뷰내용"]
# rowList = []
# eachRowList = []
# for i in range(len(reviewLists)):
#     reviewId = reviewLists[i].find(name="span",
#                    attrs={"class":reviewIdCls}).text
#     reviewContent = reviewLists[i].find(name="div",
#                        attrs={"class":reviewContentCls}).text
#     eachRowList.append(reviewId)
#     eachRowList.append(reviewContent)
#     rowList.append(eachRowList)
#     eachRowList = []

# df = pd.DataFrame(rowList,columns=columnList)
# #place별 저장
# placePath = path.join(directoryPath,placeId)
# os.makedirs(placePath, exist_ok=True)
# df.to_csv(path.join(placePath,"reviewData.csv"))    




NameError: name 'storeList' is not defined

In [ ]:
# placeId = "1194099490"
for i in range(0,len(storeList)):
    placeId = storeList[i]["placeId"]
    reviewUrl = "https://pcmap.place.naver.com/restaurant/{}/review/visitor".format(placeId)
    directoryPath = "./places"
    os.makedirs(directoryPath, exist_ok=True)
    
    #더보기 버튼
    addBtnSelector = 'a.fvwqf'
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # URL 이동
    driver.get(reviewUrl)
    
    # 웹페이지 파싱 될때까지 최대 3초 기다림
    driver.implicitly_wait(3)
    
    #더보기 xpath
    # xPath = '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div'
    # name = WebDriverWait(driver, 10).until(
    #                     EC.element_to_be_clickable(
    #                         (By.CSS_SELECTOR, addBtnSelector)
    #                     )
    #                 )
    # driver.execute_script("arguments[0].click();", name)
    # 첫 번째 버튼 찾기 및 클릭
    wait = WebDriverWait(driver, 10)  # 10초 동안 대기
    dfList = []
    storeName = storeList[i]['name']
    for j in range(0,10):
    # for _ in iter(int, 1):
        print(storeName)
        print(j,"  : 회차")
        try:
           #더보기 버튼
           # driver.find_element(By.XPATH, xPath).click()
           driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
           # addBtn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,addBtnSelector)))
           # addBtn.click()
           name = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable(
                            (By.CSS_SELECTOR, addBtnSelector)
                        )
                    )
           driver.execute_script("arguments[0].click();", name)
           #디버그
           # html = driver.page_source
           # dfList.append(debug_bsHtml(html))
        except Exception as e:
            print(e)
            print("없다")
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            #디버그
            #html = driver.page_source
            #dfList.append(debug_bsHtml(html))
            break
    
    html = driver.page_source
    bsHtml(html,directoryPath,placeId,storeName)

# BeautifulSoup로 페이지 소스 파싱
# bsObj = bs4.BeautifulSoup(html,"html.parser")

# reviewListCls = "pui__X35jYm EjjAW" 
# reviewIdCls = "pui__NMi-Dp" #span
# reviewContentCls = "pui__vn15t2" #div

# reviewLists = bsObj.findAll(name="li",
#                               attrs={"class":reviewListCls})

# columnList = ["리뷰작성ID","리뷰내용"]
# rowList = []
# eachRowList = []
# for i in range(len(reviewLists)):
#     reviewId = reviewLists[i].find(name="span",
#                    attrs={"class":reviewIdCls}).text
#     reviewContent = reviewLists[i].find(name="div",
#                        attrs={"class":reviewContentCls}).text
#     eachRowList.append(reviewId)
#     eachRowList.append(reviewContent)
#     rowList.append(eachRowList)
#     eachRowList = []

# df = pd.DataFrame(rowList,columns=columnList)
# #place별 저장
# placePath = path.join(directoryPath,placeId)
# os.makedirs(placePath, exist_ok=True)
# df.to_csv(path.join(placePath,"reviewData.csv"))    




진부령황태촌
0   : 회차
진부령황태촌
1   : 회차
진부령황태촌
2   : 회차
진부령황태촌
3   : 회차
진부령황태촌
4   : 회차
진부령황태촌
5   : 회차
진부령황태촌
6   : 회차
진부령황태촌
7   : 회차
진부령황태촌
8   : 회차
진부령황태촌
9   : 회차
화곡정육식당
0   : 회차
화곡정육식당
1   : 회차
화곡정육식당
2   : 회차
화곡정육식당
3   : 회차
화곡정육식당
4   : 회차
화곡정육식당
5   : 회차
화곡정육식당
6   : 회차
화곡정육식당
7   : 회차
화곡정육식당
8   : 회차
화곡정육식당
9   : 회차
다이닝갈비 화곡점
0   : 회차
다이닝갈비 화곡점
1   : 회차
다이닝갈비 화곡점
2   : 회차
다이닝갈비 화곡점
3   : 회차
다이닝갈비 화곡점
4   : 회차
다이닝갈비 화곡점
5   : 회차
다이닝갈비 화곡점
6   : 회차
다이닝갈비 화곡점
7   : 회차
다이닝갈비 화곡점
8   : 회차
다이닝갈비 화곡점
9   : 회차
리연탄 강서화곡본점
0   : 회차
리연탄 강서화곡본점
1   : 회차
리연탄 강서화곡본점
2   : 회차
리연탄 강서화곡본점
3   : 회차
리연탄 강서화곡본점
4   : 회차
리연탄 강서화곡본점
5   : 회차
리연탄 강서화곡본점
6   : 회차
리연탄 강서화곡본점
7   : 회차
리연탄 강서화곡본점
8   : 회차
리연탄 강서화곡본점
9   : 회차
마장동김씨 우장산역점
0   : 회차
마장동김씨 우장산역점
1   : 회차
마장동김씨 우장산역점
2   : 회차
마장동김씨 우장산역점
3   : 회차
마장동김씨 우장산역점
4   : 회차
마장동김씨 우장산역점
5   : 회차
마장동김씨 우장산역점
6   : 회차
마장동김씨 우장산역점
7   : 회차
마장동김씨 우장산역점
8   : 회차
마장동김씨 우장산역점
9   : 회차
피그백
0   : 회차
피그백
1   : 회차
피그백
2   : 회차
피그백
3   : 회차
피그백
4   : 회차
피그백
5   : 회차
피그

In [82]:
placeId = "1194099490"
reviewUrl = "https://pcmap.place.naver.com/restaurant/{}/review/visitor".format(placeId)
directoryPath = "./places"
os.makedirs(directoryPath, exist_ok=True)
#더보기 버튼
addBtnSelector = 'a.fvwqf'

In [92]:
def setup_webdriver(osType='linux'):
    options = webdriver.ChromeOptions()
    if osType == 'linux':  # 문자열로 비교
        options.add_argument("--headless")  # 헤드리스 모드
    options.add_argument("window-size=1920x1080")
    options.add_argument("--no-sandbox")  
    options.add_argument("--disable-dev-shm-usage")  
    options.add_argument("--disable-gpu")  
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")  
    options.add_argument("--disable-extensions")
    
    # User-agent 설정
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver
#    
placeId = "1194099490"
reviewUrl = "https://pcmap.place.naver.com/restaurant/{}/review/visitor".format(placeId)
directoryPath = "./places"
os.makedirs(directoryPath, exist_ok=True)
#더보기 버튼
addBtnSelector = 'a.fvwqf'
driver = setup_webdriver('window')
# URL 이동
driver.get(reviewUrl)

# 웹페이지 파싱 될때까지 최대 3초 기다림
driver.implicitly_wait(3)

#더보기 xpath
xPath = '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div'
# name = WebDriverWait(driver, 10).until(
#                     EC.element_to_be_clickable(
#                         (By.CSS_SELECTOR, addBtnSelector)
#                     )
#                 )
# driver.execute_script("arguments[0].click();", name)
# 첫 번째 버튼 찾기 및 클릭
wait = WebDriverWait(driver, 10)  # 10초 동안 대기
dfList = []
for i in range(0,10):
    print(i,"  : 회차")
    try:
       #더보기 버튼
       # driver.find_element(By.XPATH, xPath).click()
       driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
       # addBtn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,addBtnSelector)))
       # addBtn.click()
       name = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR, addBtnSelector)
                    )
                )
       driver.execute_script("arguments[0].click();", name)
       #디버그
       # html = driver.page_source
       # dfList.append(debug_bsHtml(html))
    except Exception as e:
        print(e)
        print("없다")
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        #디버그
        #html = driver.page_source
        #dfList.append(debug_bsHtml(html))
        break

html = driver.page_source
result = debug_bsHtml(html)



# BeautifulSoup로 페이지 소스 파싱
# bsObj = bs4.BeautifulSoup(html,"html.parser")

# reviewListCls = "pui__X35jYm EjjAW" 
# reviewIdCls = "pui__NMi-Dp" #span
# reviewContentCls = "pui__vn15t2" #div

# reviewLists = bsObj.findAll(name="li",
#                               attrs={"class":reviewListCls})

# columnList = ["리뷰작성ID","리뷰내용"]
# rowList = []
# eachRowList = []
# for i in range(len(reviewLists)):
#     reviewId = reviewLists[i].find(name="span",
#                    attrs={"class":reviewIdCls}).text
#     reviewContent = reviewLists[i].find(name="div",
#                        attrs={"class":reviewContentCls}).text
#     eachRowList.append(reviewId)
#     eachRowList.append(reviewContent)
#     rowList.append(eachRowList)
#     eachRowList = []

# df = pd.DataFrame(rowList,columns=columnList)
# #place별 저장
# placePath = path.join(directoryPath,placeId)
# os.makedirs(placePath, exist_ok=True)
# df.to_csv(path.join(placePath,"reviewData.csv"))    




0   : 회차
1   : 회차
2   : 회차
3   : 회차
4   : 회차
5   : 회차
6   : 회차
7   : 회차
8   : 회차
9   : 회차


In [115]:
result

,리뷰작성ID,리뷰내용,리뷰일시
0,찡보 D,겉에 분위기부터 막막 들어가고 싶었는데 안에 인테리어도 짱입니당!! 모츠나베먹고싶어...,9.19.목
1,찔끄밍50,모츠나베 비주얼 일단 너무 좋아서 놀랬고 맛도 맛있네요~🩵까치산에 이런 곳 생겨서 ...,9.20.금
2,non****,최근에 새로 생겨서 한 번 와보고 싶었는데 음식도 맛있고 분위기도 좋고 사장님들도 ...,7.30.화
3,12y****,"찾아보고 왔는데 진짜 찐 나만 알고싶은,, 맛집이다유,,🥺💜 안주 다 맛있고 진짜 ...",8.25.일
4,ga****,동네에 새로 생긴 이자카야 처음 와봤는데 분위기도 넘 좋고 안주들도 맛있어요! 까치...,9.22.일
5,012****,분위기도 정말 좋고 음식이 너므너무너무너무 맛있어요ㅜㅜ 제가 사진은 못찍었는데 모츠...,7.28.일
6,뚜뚜비75,진짜 너무너무 맛있어요 안주들이 다 특색있어요 다른 이자카야에서도 있는 메뉴들도 있...,7.28.일
7,후민94,안주가 전부 맛있어요 육회가 정말 별미예요~! 파밥도 사이드인데 너무 맛있어요,8.16.금
8,민정7722,직원들도 친절하고 음식도 너무맛나옹🩵담에 또올거에옹💗🙏,9.8.일
9,땡글맘48,지나가다 분위기있어보여 들어왔네요.분위기 좋고 직원분이 많이친절하시고~ 음식...,8.3.토


In [93]:
result

,리뷰작성ID,리뷰내용


In [86]:
# BeautifulSoup로 페이지 소스 파싱
def bsHtml(html,directoryPath,placeId,storeName):
    bsObj = bs4.BeautifulSoup(html,"html.parser")
    
    reviewListCls = "pui__X35jYm EjjAW" 
    reviewIdCls = "pui__NMi-Dp" #s|pan
    reviewContentCls = "pui__vn15t2" #div
    
    reviewLists = bsObj.findAll(name="li",
                                  attrs={"class":reviewListCls})
    
    columnList = ["리뷰작성ID","리뷰내용"]
    rowList = []
    eachRowList = []
    for i in range(len(reviewLists)):
        reviewId = reviewLists[i].find(name="span",
                       attrs={"class":reviewIdCls}).text
        reviewContent = reviewLists[i].find(name="div",
                           attrs={"class":reviewContentCls}).text
        eachRowList.append(reviewId)
        eachRowList.append(reviewContent)
        rowList.append(eachRowList)
        eachRowList = []
    
    df = pd.DataFrame(rowList,columns=columnList).reset_index(drop=True)
    df['placeId'] = placeId
    df['상점이름'] = storeName
    #place별 저장
    placePath = os.path.join(directoryPath,placeId)
    os.makedirs(placePath, exist_ok=True)
    df.to_csv(os.path.join(placePath,"reviewData.csv"))    
    return "success"

In [112]:
## debug용
def debug_bsHtml(html):
    bsObj = bs4.BeautifulSoup(html,"html.parser")
    
    reviewListCls = "pui__X35jYm" 
    reviewIdCls = "pui__NMi-Dp" #span
    reviewContentCls = "pui__vn15t2" #div
    
    reviewLists = bsObj.findAll(name="li",
                                  attrs={"class":reviewListCls})
    
    columnList = ["리뷰작성ID","리뷰내용","리뷰일시"]
    rowList = []
    eachRowList = []
    for i in range(len(reviewLists)):
        reviewId = reviewLists[i].find(name="span",
                       attrs={"class":reviewIdCls}).text
        reviewContent = reviewLists[i].find(name="div",
                           attrs={"class":reviewContentCls}).text
        reviewDate = reviewDate = reviewLists[i].find(name="time").text
        
        eachRowList.append(reviewId)
        eachRowList.append(reviewContent)
        eachRowList.append(reviewDate)
        
        rowList.append(eachRowList)
        eachRowList = []
    df = pd.DataFrame(rowList,columns=columnList)
    #place별 저장
    # placePath = path.join(directoryPath,placeId)
    # os.makedirs(placePath, exist_ok=True)
    # df.to_csv(path.join(placePath,"reviewData.csv"))    
    return df


## beautifulSoup로 데이터 정제

In [23]:
# BeautifulSoup로 페이지 소스 파싱
bsObj = bs4.BeautifulSoup(html,"html.parser")

reviewListCls = "pui__X35jYm EjjAW" 
reviewIdCls = "pui__NMi-Dp" #span
reviewContentCls = "pui__vn15t2" #div

reviewLists = bsObj.findAll(name="li",
                              attrs={"class":reviewListCls})

columnList = ["리뷰작성ID","리뷰내용"]
rowList = []
eachRowList = []
for i in range(len(reviewLists)):
    reviewId = reviewLists[i].find(name="span",
                   attrs={"class":reviewIdCls}).text
    reviewContent = reviewLists[i].find(name="div",
                       attrs={"class":reviewContentCls}).text
    eachRowList.append(reviewId)
    eachRowList.append(reviewContent)
    rowList.append(eachRowList)
    eachRowList = []

df = pd.DataFrame(rowList,columns=columnList)
df.to_csv("./reviewData.csv")    

In [26]:
reviewId = reviewLists[0].find(name="span",
                   attrs={"class":reviewIdCls}).text
reviewContent = reviewLists[0].find(name="div",
                   attrs={"class":reviewContentCls}).text


별리미97 화곡사는 친구가 정말 너무너무너무!!맛있다길래 족발이 맛있어봤자라고 생각하고 온건데,,ㄹㅇ쫀맛이에욬ㅋㅋㅋㅋㅋ 진짜 인생족발,, 껍데기도 살도 부드러운게 차원이 달라요ㅠㅠㅠ 날이 엄청 더웠는데 포장 웨이팅도 있었는데 ,, 다 이유가 있었어요 개맛도리 족발 벌써 또 먹구싶어용❤️❤️


In [29]:
import pandas as pd

In [34]:
columnList = ["리뷰작성ID","리뷰내용"]
rowList = []
eachRowList = []
for i in range(len(reviewLists)):
    reviewId = reviewLists[i].find(name="span",
                   attrs={"class":reviewIdCls}).text
    reviewContent = reviewLists[i].find(name="div",
                       attrs={"class":reviewContentCls}).text
    eachRowList.append(reviewId)
    eachRowList.append(reviewContent)
    rowList.append(eachRowList)
    eachRowList = []

df = pd.DataFrame(rowList,columns=columnList)
df.to_csv("./reviewData.csv")    





In [21]:
reviewImgTag[0]

<div class="lazyload-wrapper"><div class="lazyload-placeholder"></div></div>

In [80]:
df = pd.read_csv("./places/1194099490/reviewData.csv")

In [88]:
print(df.head())

   Unnamed: 0   리뷰작성ID                                               리뷰내용
0           0  non****  최근에 새로 생겨서 한 번 와보고 싶었는데 음식도 맛있고 분위기도 좋고 사장님들도 ...
1           1  12y****  찾아보고 왔는데 진짜 찐 나만 알고싶은,, 맛집이다유,,🥺💜 안주 다 맛있고 진짜 ...
2           2  012****  분위기도 정말 좋고 음식이 너므너무너무너무 맛있어요ㅜㅜ 제가 사진은 못찍었는데 모츠...
3           3    뚜뚜비75  진짜 너무너무 맛있어요 안주들이 다 특색있어요 다른 이자카야에서도 있는 메뉴들도 있...
4           4     후민94        안주가 전부 맛있어요 육회가 정말 별미예요~! 파밥도 사이드인데 너무 맛있어요
